In [30]:
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
from terminaltables import AsciiTable
from scipy.stats import chi2

In [31]:
#SE IMPORTA EL CSV
honda_data = pd.read_csv('Honda2.csv')
honda_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-01,21.190001,21.629999,21.030001,21.080000,19.664743,900200
1,2020-04-02,20.559999,20.990000,20.430000,20.879999,19.478172,1213100
2,2020-04-03,20.000000,20.120001,19.750000,19.860001,18.526655,1424600
3,2020-04-06,21.090000,21.559999,20.870001,21.480000,20.037889,1642100
4,2020-04-07,22.549999,22.719999,21.690001,21.700001,20.243120,1246700
...,...,...,...,...,...,...,...
980,2024-02-23,35.790001,35.900002,35.680000,35.820000,35.820000,756400
981,2024-02-26,35.689999,35.820000,35.580002,35.660000,35.660000,698100
982,2024-02-27,35.500000,35.500000,35.169998,35.250000,35.250000,695600
983,2024-02-28,35.279999,35.490002,35.209999,35.369999,35.369999,503000


In [32]:
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
honda_data = honda_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
honda_data

,Date,Adj Close
0,2020-04-01,19.664743
1,2020-04-02,19.478172
2,2020-04-03,18.526655
3,2020-04-06,20.037889
4,2020-04-07,20.243120
...,...,...
980,2024-02-23,35.820000
981,2024-02-26,35.660000
982,2024-02-27,35.250000
983,2024-02-28,35.369999


In [33]:
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(honda_data)):
    percent = (honda_data['Adj Close'][i] - honda_data['Adj Close'][i-1])/honda_data['Adj Close'][i-1]
    percentage.append(percent)
honda_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
honda_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
honda_data.insert(4, 'Current State', states)

#SE ASIGNA UNA NUEVA COLUMNA CON EL ESTADO SIGUIENTE AL REGISTRO
next_state_column=[]
for i in range(len(honda_data)-1):
    next_state_column.append(honda_data['Current State'][i+1])

previows_state_column=[0]
for i in range(1,len(honda_data)-1):
    previows_state_column.append(honda_data['Current State'][i-1])
    
#SE BORRA LA ULTIMA FILA POR QUE NO TIENE ESTADO SIGUIENTE
honda_data = honda_data.drop(honda_data.index[-1])
honda_data.insert(5, 'Next State', next_state_column)
honda_data.insert(4, 'Previous State', previows_state_column)

complete_table = AsciiTable([honda_data.columns]+list(honda_data.values))
print("Complete Table \n",complete_table.table)

Complete Table 
 +------------+-----------+-------------------------+----------------------+----------------+---------------+------------+
| Date       | Adj Close | Change %                | std dev              | Previous State | Current State | Next State |
+------------+-----------+-------------------------+----------------------+----------------+---------------+------------+
| 2020-04-01 | 19.664743 | 0.0                     | 0.0                  | 0              | 0             | 0          |
| 2020-04-02 | 19.478172 | -0.009487589031801772   | 0.0                  | 0              | 0             | 0          |
| 2020-04-03 | 18.526655 | -0.0488504260050686     | 0.0                  | 0              | 0             | 0          |
| 2020-04-06 | 20.037889 | 0.08157079623925625     | 0.0                  | 0              | 0             | 0          |
| 2020-04-07 | 20.24312  | 0.010242146765060994    | 0.0                  | 0              | 0             | 0          |
| 2020-

In [34]:
#SE QUITAN LAS PRIMERAS 20 FILAS, PUES ESTAS NO CUENTAN CON ESTADO ACTUAL
honda_trim=honda_data.iloc[21:]
table_week_21 = AsciiTable([honda_trim.columns]+list(honda_trim.values))
print("Table since week 21 \n",table_week_21.table)

Table since week 21 
 +------------+-----------+-------------------------+----------------------+----------------+---------------+------------+
| Date       | Adj Close | Change %                | std dev              | Previous State | Current State | Next State |
+------------+-----------+-------------------------+----------------------+----------------+---------------+------------+
| 2020-05-01 | 21.39987  | -0.04615382146752623    | 0.033547964908324394 | S1             | B2            | B1         |
| 2020-05-04 | 21.18531  | -0.010026229131298402   | 0.031415784221130984 | B2             | B1            | S1         |
| 2020-05-05 | 21.334568 | 0.007045353596430718    | 0.026392964624180438 | B1             | S1            | B1         |
| 2020-05-06 | 21.315914 | -0.0008743556466670196  | 0.02636181064778357  | S1             | B1            | B1         |
| 2020-05-07 | 21.259941 | -0.002625878486843119   | 0.02609535511382918  | B1             | B1            | S2         |
| 

In [39]:
def validate_markov_rule(data):
    
    #PROBABILIDAD OBSERVADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    states_before = list(itertools.product(states,repeat=2))
    states_mapping_before = {}
    for i,state_before in enumerate(states_before):
        complete_state=f"{state_before[0]}-{state_before[1]}"
        states_mapping_before[complete_state]=i

    transitions_observed=np.zeros(((len(states_mapping_before)),len(states_mapping)))
    freq_observed=np.zeros(((len(states_mapping_before)),len(states_mapping)))
    for i in range(len(data)):
        freq_observed[states_mapping_before[data.iloc[i]['Previous State']+"-"+data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    for i in range(len(transitions_observed)):
        if(sum(freq_observed[i])!=0):
            transitions_observed[i]=freq_observed[i]/sum(freq_observed[i])
        else:
            freq_suavizada=1e-100
            transitions_observed[i]=freq_observed[i]/freq_suavizada

    data_frame_observed=pd.DataFrame(transitions_observed,columns=list(states_mapping.keys()),index=list(states_mapping_before.keys()))
    print(f"Probabilidad Observada \n {data_frame_observed}\n")

    #PROBABILIDAD ESPERADA
    transition_expected = np.zeros(((len(states_mapping)), len(states_mapping)))
    for i in range(len(data)):
        transition_expected[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    for i in range(len(transition_expected)):
        if sum(transition_expected[i])!=0:
            transition_expected[i] = transition_expected[i] / sum(transition_expected[i])

    data_frame_expected = pd.DataFrame(transition_expected, columns=list(states_mapping.keys()), index=(list(states_mapping.keys())))
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")
    transition_expected=np.tile(transition_expected,(len(transition_expected),1))

    #CHI CUADRADO
    transition_expected = np.where(transition_expected == 0, 1e-100, transition_expected)
    chi2_observed=(np.square(transitions_observed-transition_expected))/transition_expected
    for i in range(len(chi2_observed)):
        chi2_observed[i]=chi2_observed[i]*sum(freq_observed[i])
    chi2_observed=np.sum(chi2_observed)

    #VALORES NECESARIOS
    rows,columns=transitions_observed.shape
    grades=(rows-1)*(columns-1)
    alpha=0.05
    chi2_theory=chi2.ppf(1-alpha,grades)

    print(f"Número de Filas: {rows}")
    print(f"Número de Columnas: {columns}")
    print(f"Grados de Libertad: {grades}")
    print(f"Porcentaje de Confianza: {(1-alpha)*100}%")
    print(f"Chi Cuadrado: {chi2_observed}")
    print(f"Chi Teórico: {chi2_theory}")
    print(f"¿Se Acepta?: {chi2_observed<chi2_theory}")


validate_markov_rule(honda_trim[['Previous State', 'Current State', 'Next State']])

Probabilidad Observada 
              B1        B2        B3        S1        S2        S3
B1-B1  0.278846  0.115385  0.028846  0.442308  0.105769  0.028846
B1-B2  0.281250  0.125000  0.031250  0.500000  0.062500  0.000000
B1-B3  0.428571  0.000000  0.142857  0.142857  0.142857  0.142857
B1-S1  0.333333  0.166667  0.017544  0.333333  0.131579  0.017544
B1-S2  0.342857  0.028571  0.000000  0.485714  0.085714  0.057143
B1-S3  0.400000  0.066667  0.000000  0.466667  0.066667  0.000000
B2-B1  0.285714  0.178571  0.035714  0.250000  0.142857  0.107143
B2-B2  0.130435  0.130435  0.086957  0.391304  0.260870  0.000000
B2-B3  0.200000  0.000000  0.000000  0.400000  0.400000  0.000000
B2-S1  0.212766  0.106383  0.042553  0.404255  0.234043  0.000000
B2-S2  0.400000  0.200000  0.000000  0.200000  0.133333  0.066667
B2-S3  1.000000  0.000000  0.000000  0.000000  0.000000  0.000000
B3-B1  0.285714  0.428571  0.000000  0.142857  0.142857  0.000000
B3-B2  0.000000  0.000000  0.000000  0.000000  1.00

In [36]:
def markov_model(data):
    #PROBABILIDAD OBSERVADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #PROBABILIDAD ESPERADA
    transition_matrix = np.zeros(((len(states_mapping)), len(states_mapping)))
    for i in range(len(data)):
        transition_matrix[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    for i in range(len(transition_matrix)):
        transition_matrix[i] = transition_matrix[i] / sum(transition_matrix[i])
    data_frame_expected = pd.DataFrame(transition_matrix, columns=list(states_mapping.keys()), index=list(states_mapping.keys()))
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")

    return transition_matrix
#SE ENVIAN LAS COLUMNAS DE ESTADO ACTUAL Y SIGUIENTE
transition_matrix = markov_model(honda_trim[['Current State', 'Next State']])

Probabilidad Esperada
          B1        B2        B3        S1        S2        S3
B1  0.338762  0.104235  0.022801  0.371336  0.114007  0.048860
B2  0.235294  0.193277  0.042017  0.394958  0.126050  0.008403
B3  0.280000  0.080000  0.040000  0.320000  0.200000  0.080000
S1  0.319444  0.125000  0.022222  0.361111  0.152778  0.019444
S2  0.352459  0.106557  0.032787  0.393443  0.081967  0.032787
S3  0.333333  0.100000  0.000000  0.466667  0.066667  0.033333

